# Многопоточность

Мы рассмотрели базовые принципы и типичные проблемы параллельного выполнения программ. Основной проблемой является синхронизация потоков. В этом уроке мы рассмотрим примитивы синхронизации.

## Примитивы синхронизации

Синхронизация потоков - это согласованное выполнение потоков. Синхронизация потоков необходима для обеспечения предскауемого результата выполнения операций. Например, если один поток пишет в файл, то другой поток не должен читать из этого файла. Для синхронизации потоков используются примитивы синхронизации.

Одни из самых простых примитивов синхронизации - это мьютексы. Мьютекс - это объект, который может находиться в двух состояниях: заблокированном и разблокированном. Поток может заблокировать мьютекс, если он свободен. Если мьютекс уже заблокирован, то поток, который пытается его заблокировать, будет ожидать, пока мьютекс не будет разблокирован. Мьютексы используются для синхронизации доступа к ресурсам, которые не могут быть использованы несколькими потоками одновременно. Они похожи на `Monitor` в C#, однако мьютексы могут быть использованы для синхронизации потоков в разных процессах.

In [ ]:
using System;
using System.Threading;

static void Main()
{
    // Naming a Mutex makes it available computer-wide. Use a name that's
    // unique to your company and application (e.g., include your URL).

    using (var mutex = new Mutex(false, "MyApp.Multithreading"))
    {
        // Wait a few seconds if contended, in case another instance
        // of the program is still in the process of shutting down.

        if (!mutex.WaitOne(TimeSpan.FromSeconds(3), false))
        {
            Console.WriteLine("Another app instance is running. Bye!");
            return;
        }
        
        RunProgram();
    }
}

static void RunProgram()
{
    Console.WriteLine("Running. Press Enter to exit");
    Console.ReadLine();
}

Чтобы Мьютекс был доступен во всей системе, его нужно назвать. Используйте имя, которое уникально для вашей компании и приложения (например, включите в него ваш URL).

Другой похожей примитивой синхронизации является семафор. Семафор работает как "ночной клуб": у него есть определенная вместимость, и если все места заняты, то новые посетители должны ждать, пока кто-то не покинет клуб. Семафоры используются для ограничения количества потоков, которые могут выполнять некоторую операцию одновременно. Например, если у вас есть 4 ядра процессора, то вы можете создать семафор с вместимостью 4, и только 4 потока смогут выполнять некоторую операцию одновременно.

Семафор с вместимостью 1 работает так же как `Mutex`, за исключением того, что семафор может быть разблокирован любым потоком, а не только тем, который его заблокировал.

In [1]:
using System;
using System.Threading;

static SemaphoreSlim _sem = new SemaphoreSlim(3);    // Capacity of 3

static void Main()
{
    var threads = new Thread[5];
    for (int i = 0; i < 5; i++)
    {
        threads[i] = new Thread(Enter);
        threads[i].Start(i + 1);
    }
}

static void Enter(object id)
{
    Console.WriteLine(id + " wants to enter");
    _sem.Wait();
    Console.WriteLine(id + " is in!");           // Only three threads
    Thread.Sleep(1000 * (int)id);               // can be here at
    Console.WriteLine(id + " is leaving");     // a time.
    _sem.Release();
}

Main();

1 wants to enter
2 wants to enter
2 is in!
4 wants to enter
4 is in!
1 is in!
3 wants to enter
5 wants to enter


## Неизменяемые объекты

Другим важным инструментом многопоточного программирования являются неизменяемые объекты. Неизменяемый объект - это объект, который не может быть изменен после создания. Неизменяемые объекты очень полезны, потому что они не нуждаются в синхронизации. Неизменяемые объекты могут быть использованы несколькими потоками одновременно без каких-либо проблем.

В `dotnet` есть несколько типов, которые являются неизменяемыми. Например, все значимые типы, делегаты и тип `string` являются неизменяемыми. Неизменяемость - это отличительная черта функционального программирования, где вместо изменения объектов создаются новые объекты.


In [2]:
using System;
using System.Threading;

public static void Main()
{            
    // how to fix?
            
    var hello = new char [] {'H', 'e', 'l', 'l', 'o'};
    var thread = new Thread(() => Console.WriteLine((hello.ToUpper())));
    thread.Start();
    Console.WriteLine(hello.ToLower());
    thread.Join();
}

private static char[] ToUpper(this char[] obj)
{
    for (int i = 0; i < obj.Length; i++)
    {
        obj[i] = char.ToUpper(obj[i]);
    }

    return obj;
}

private static char[] ToLower(this char[] obj)
{
    for (int i = 0; i < obj.Length; i++)
    {
        obj[i] = char.ToLower(obj[i]);
    }

    return obj;
}

Main();

HELLO
HELLO


В `dotnet` вы сможете найти разные примитивы синхронизации, одной из отличительных особенностей будет являться скорость работы и затратность ресурсов. Наиболее эффективныит будут примитивы в постфиксом `Slim`, например `SemaphoreSlim` или `MutexSlim`. Они используют меньше ресурсов, чем их старшие братья, но они не могут быть использованы для синхронизации между процессами.

## Утлизация потоков

Потоки - это дорогой ресурс, поэтому важно использовать их эффективно. Если вы создаете много потоков, то можете столкнуться с проблемой, когда ваша программа будет тратить больше времени на переключение контекста между потоками, чем на полезную работу. Поэтому важно использовать потоки только тогда, когда это необходимо.

In [ ]:
using System;
using System.Threading;
using System.Diagnostics;

static void Main()
{          
    Console.WriteLine("Hello world!");
    int i = 1;
    while(true)
    {
        new Thread(obj =>
        {
            Thread.Sleep(500);
        }).Start();

        Console.WriteLine($"{i++}: {Process.GetCurrentProcess().PagedMemorySize64 / 1024 / 1024}MB");
    }
}

Main();

В данном примере мы сэмулировали ситуацию, когда операция, которую необходиом выполнить параллельно, занимает 500 мс. В таком случае, если мы будем создавать потоки для каждой операции, то будем использовать больше ресурсов, чем это необохдимо.

## Пул потоков

Для того, чтобы избежать создания большого количества потоков, в `dotnet` есть пул потоков. Пул потоков - это набор потоков, которые могут быть использованы для выполнения задач. Пул потоков создается при запуске приложения и уничтожается при его завершении. Пул потоков автоматически управляет количеством потоков, которые он содержит, и может быть настроен для выполнения задач с различными приоритетами.

In [ ]:
using System;
using System.Threading;
using System.Diagnostics;

static void Main(string[] args)
{          
    Console.WriteLine("Hello world!");
    int i = 1;
    while(true)
    {
        ThreadPool.QueueUserWorkItem(obj =>
        {
            Thread.Sleep(500);
        });

        Console.WriteLine($"{i++}: {Process.GetCurrentProcess().PagedMemorySize64 / 1024 / 1024}MB");
    }
}

На примере выше мы можем увидеть, что пул потоков использует меньше ресурсов, чем создание потоков вручную. При обработке большого количества задач, пул потоков может быть более эффективным, чем создание потоков вручную, при этом для таких задач существует такое понятие как `пропускная способность` (*throughput*). Пропускная способность - это количество задач, которые могут быть обработаны за единицу времени.

Давайте посмотрим что это означает на примере:

In [ ]:
using System;
using System.Threading;
using System.Diagnostics;

private static int counter;
private const int OperationDuration = 2000;

private static int MaxThreads
{
	get
	{
		ThreadPool.GetMaxThreads(out var workerThreads, out _);
		return workerThreads;
	}
}
private static int AvailableThreads
{
	get
	{
		ThreadPool.GetAvailableThreads(out var workerThreads, out _);
		return workerThreads;
	}
}

static void Main(string[] args)
{
	ThreadPool.SetMaxThreads(10, 10);
	Console.WriteLine($"Starting with thread capacity {MaxThreads}");
	var timer = new Timer(TimerTick);

	Console.WriteLine("Performing operation...");
	var sw = new Stopwatch();
	sw.Start();
	DoOperationAsync().Wait();
	sw.Stop();
	Console.WriteLine($"Single operation completes in {sw.ElapsedMilliseconds} ms.");

	timer.Change(0, 1000);
	Thread.Sleep(10000);
	Console.WriteLine($"Operations completed: {counter}");
}

private static void TimerTick(object state)
{
	for (int i = 0; i < 100; i++)
	{
		DoOperationAsync();
	}
	Console.WriteLine($"Added another 100 operations. Threads available {AvailableThreads}");
}

private static async Task DoOperationAsync()
{
	await Task.Yield();
	Thread.Sleep(OperationDuration);
	//await Task.Delay(OperationDuration);
	Interlocked.Increment(ref counter);
}